In [44]:
# The links to the dataset can be found here:
# https://www.kaggle.com/datasets/jillanisofttech/updated-resume-dataset
# https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset
# https://www.kaggle.com/datasets/arshkon/linkedin-job-postings

# I combined the resume-dataset and updated-resume-dataset into resumes.csv.

In [45]:
%load_ext memory_profiler
%memit
import os
import sys
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow as tf
import logging
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import ssl
import nltk
import hashlib
import json
import time
import asyncio
import aiohttp
from openai import AsyncOpenAI

client = AsyncOpenAI(api_key="sk-proj-sScFMZr4GK3XaQ5mcYFYT3BlbkFJ1orIjm1gKSNT0q30KBMZ")

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 397.77 MiB, increment: 0.47 MiB


In [46]:
%%memit
postings = pd.read_csv("data/trunc-postings.csv")
resumes = pd.read_csv("data/trunc-resumes.csv")
postings = postings.head(3)
resumes = resumes.head(3)

peak memory: 413.89 MiB, increment: 13.95 MiB


In [48]:
%%memit
if os.path.exists('cache.json'):
	with open('cache.json', 'r') as cache_file:
		cache = json.load(cache_file)
else:
	print("ERROR: Could not find cache. Aborting")
	sys.exit(1)


labels = []
for job_description in postings['description']:
	for resume_text in resumes['Resume']:
		key = hashlib.md5(f"{resume_text}{job_description}".encode()).hexdigest()
		relevance = cache.get(key)
		if relevance == None:
			print(f"ERROR: COULD NOT FIND LABEL FOR KEY {key}. THIS SHOULD BE ADDRESSED. DEFAULTING TO 0")
			relevance = 0
		labels.append(relevance)

print(labels)

[0, 0, 0, 1, 1, 1, 0, 0, 0]
peak memory: 396.31 MiB, increment: 1.70 MiB


In [49]:
%%memit

nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text_series):
    logging.info("Preprocessing text...")
    text_series = text_series.fillna("")  # Replace NaN with empty strings
    text_series = text_series.apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()))
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    text_series = text_series.apply(lambda x: ' '.join(
        lemmatizer.lemmatize(word) for word in x.split() if word not in stop_words
    ))
    return text_series

resumes['Resume'] = preprocess_text(resumes['Resume'])
postings['description'] = preprocess_text(postings['description'])

display(resumes.head(3))
display(postings.head(3))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/armandrismir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/armandrismir/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Resume,Category
0,hr administratormarketing associate hr adminis...,HR
1,hr specialist u hr operation summary versatile...,HR
2,hr director summary 20 year experience recruit...,HR


,description
0,job descriptiona leading real estate firm new ...
1,aspen therapy wellness committed serving clien...
2,national exemplar accepting application assist...


peak memory: 397.67 MiB, increment: 4.48 MiB


In [50]:
%%memit
universal_sentence_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def vectorize_text(text_series):
    logging.info("Vectorizing text...")
    embeddings = universal_sentence_encoder(text_series.tolist()).numpy()
    return embeddings

vectorized_resumes = vectorize_text(resumes['Resume'])
vectorized_postings = vectorize_text(postings['description'])

print(f'vectorized resumes: {type(vectorized_resumes)}, {vectorized_resumes.nbytes}')
print(f'vectorized postings: {type(vectorized_postings)}, {vectorized_postings.nbytes}')

vectorized resumes: <class 'numpy.ndarray'>, 6144
vectorized postings: <class 'numpy.ndarray'>, 6144
peak memory: 875.00 MiB, increment: 477.66 MiB


In [51]:
%%memit
resume_posting_pairs = [(vectorized_resumes[i], vectorized_postings[j])
                for i in range(len(resumes)) for j in range(len(postings))]
# Ensure vector pairs are numpy arrays
resume_posting_pairs = [(np.array(left), np.array(right)) for left, right in resume_posting_pairs]

print(f'resume_posting_pairs: {type(resume_posting_pairs)}, {len(resume_posting_pairs)}')

resume_posting_pairs: <class 'list'>, 9
peak memory: 527.52 MiB, increment: -2.62 MiB


# ML Model (needs nvidia driver)
device = torch.device('cuda'))
device = torch.device('cuda')
train_pairs, test_pairs, train_labels, test_labels = train_test_split(resume_posting_pairs, labels, test_size=0.2, random_state=42)
logging.info("Data split into training and testing sets.")
print("Training and testing data prepared.")

train_left = np.array([x[0] for x in train_pairs])
train_right = np.array([x[1] for x in train_pairs])
test_left = np.array([x[0] for x in test_pairs])
test_right = np.array([x[1] for x in test_pairs])

train_data = (torch.tensor(train_left, dtype=torch.float32), torch.tensor(train_right, dtype=torch.float32))
test_data = (torch.tensor(test_left, dtype=torch.float32), torch.tensor(test_right, dtype=torch.float32))

train_labels = torch.tensor(train_labels, dtype=torch.float32).unsqueeze(1)
test_labels = torch.tensor(test_labels, dtype=torch.float32).unsqueeze(1)

print("Train Data Shape:", (train_left.shape, train_right.shape))
print("Train Data Type:", (type(train_left), type(train_right)))
print("Train Labels Shape:", len(train_labels))
print("Train Labels Type:", type(train_labels))
print("Test Data Shape:", (test_left.shape, test_right.shape))
print("Test Data Type:", (type(test_left), type(test_right)))
print("Test Labels Shape:", len(test_labels))
print("Test Labels Type:", type(test_labels))

class SiameseNetwork(nn.Module):
    def __init__(self, input_size):
        super(SiameseNetwork, self).__init__()
        self.shared_network = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.output_layer = nn.Linear(64, 1)

    def forward_one_side(self, x):
        return self.shared_network(x)

    def forward(self, input1, input2):
        output1 = self.forward_one_side(input1)
        output2 = self.forward_one_side(input2)
        distance = torch.abs(output1 - output2)
        output = torch.sigmoid(self.output_layer(distance))
        return output

input_size = train_left.shape[1]

model = SiameseNetwork(input_size).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
batch_size = 8

train_dataset = torch.utils.data.TensorDataset(train_data[0], train_data[1], train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = torch.utils.data.TensorDataset(test_data[0], test_data[1], test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

best_val_loss = float('inf')
patience_counter = 0

logging.info("Starting model training...")
print("Starting model training...")

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch_idx, (left, right, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(left.to(device), right.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, (left, right, labels) in enumerate(test_loader):
            outputs = model(left.to(device), right.to(device))
            loss = criterion(outputs, labels.to(device))
            val_loss += loss.item()

    val_loss /= len(test_loader)

    logging.info(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss}, Validation Loss: {val_loss}")
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss}, Validation Loss: {val_loss}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'siamese_network.pth')
        logging.info("Model saved.")
        print("Model saved.")
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            logging.info("Early stopping triggered.")
            print("Early stopping triggered.")
            break

logging.info("Model training completed.")
print("Model training completed.")

model.load_state_dict(torch.load('siamese_network.pth', map_location=device))
logging.info("Best model loaded.")
print("Best model loaded.")